In [104]:
import pandas as pd

df = pd.read_pickle('../data/clean_df.pkl')


In [105]:
drop_col = ['No_disposition', 'Section', 'No_plan','Nature_culture','Departement','longitude','latitude','Code_postal']


df = df.drop(columns=drop_col)

# print(df_without_nan.info())
print(df.columns)
df.head()


Index(['Valeur_fonciere', 'Commune', 'Nombre_de_lots', 'Code_type_local',
       'Surface_reelle_bati', 'Nombre_pieces_principales', 'Surface_terrain',
       'day', 'month', 'year'],
      dtype='object')


,Valeur_fonciere,Commune,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,day,month,year
2,45000.0,SAINT DENIS,1,2,20.0,1.0,NaN,22,8,2018
3,100000.0,HYERES,2,2,52.0,3.0,NaN,13,3,2018
4,102000.0,ORCHIES,0,1,54.0,2.0,103.0,12,3,2018
5,297000.0,VINCENNES,2,2,46.0,3.0,NaN,28,9,2018
6,74000.0,NANTES,2,2,36.0,2.0,NaN,9,6,2018


In [106]:
# # Encoder données
# from sklearn.preprocessing import LabelEncoder

# colonnes_catégorielles = list(
#     df_without_nan.select_dtypes(include=['object']).columns)

# print(colonnes_catégorielles)

# label_encoder = LabelEncoder()

# for col in colonnes_catégorielles:
#     df_without_nan[col] = label_encoder.fit_transform(df_without_nan[col])

##df_without_nan = pd.get_dummies(df_without_nan, columns=colonnes_non_numeriques.columns)

#df_without_nan = pd.get_dummies(
#    df_without_nan, columns=['Commune'])


# print(df_without_nan.info())
# df_without_nan.head()


In [107]:
X = df.drop(columns=['Code_type_local'])
y = df['Code_type_local']
print(y)

2          2
3          2
4          1
5          2
6          2
          ..
6521190    2
6521191    1
6521194    1
6521195    4
6521197    1
Name: Code_type_local, Length: 4084994, dtype: int32


In [108]:
# # SÉLECTION DU MEILLEUR NOMBRE DE VARIABLES 

# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import cross_val_score
# import numpy as np

# X = df.drop(columns=['Code_type_local'])
# y = df['Code_type_local']

# # Supposons que X contient vos caractéristiques et y contient votre variable cible
# # Créez un modèle DecisionTreeClassifier
# modele_decision_tree = DecisionTreeClassifier()

# # Initialisez une liste pour stocker les scores de cross-validation
# scores = []

# # Essayez différentes valeurs de k et effectuez une validation croisée pour chaque valeur de k
# # Essayez toutes les valeurs de k de 1 à nombre total de caractéristiques
# for k in range(1, X.shape[1] + 1):
#     k_best = SelectKBest(score_func=chi2, k=k)
#     X_selected = k_best.fit_transform(X, y)
#     # Utilisez une validation croisée avec 5 plis
#     score = np.mean(cross_val_score(modele_decision_tree, X_selected, y, cv=5))
#     scores.append(score)

# # Trouvez la valeur de k qui donne le meilleur score
# # Ajoutez 1 car l'indexation commence à partir de 0
# meilleur_k = np.argmax(scores) + 1
# # Obtenez le meilleur score correspondant à la meilleure valeur de k
# meilleur_score = scores[meilleur_k - 1]

# print("Meilleur nombre de caractéristiques : {}".format(meilleur_k))
# print("Meilleur score de cross-validation : ",meilleur_score )


In [109]:
# SÉLECTION DU MEILLEUR NOMBRE DE VARIABLES

# from sklearn.feature_selection import SelectKBest, chi2

# # Créez un objet SelectKBest avec le meilleur nombre de caractéristiques trouvé précédemment
# k_best = SelectKBest(score_func=chi2, k=5)
# X_selected = k_best.fit_transform(X, y)

# # Obtenez les indices des caractéristiques sélectionnées
# indices_caracteristiques_selectionnees = k_best.get_support()

# # Obtenez les noms des caractéristiques sélectionnées à partir du DataFrame original
# caracteristiques_selectionnees = X.columns[indices_caracteristiques_selectionnees]

# # Affichez les noms des caractéristiques sélectionnées
# print("Caractéristiques sélectionnées : ")
# print(caracteristiques_selectionnees)


In [110]:
# CLASSIFICATION TRAINING 'Code_type_local'


# Import train_test_split function
from sklearn.model_selection import train_test_split
# ########### Random forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Random forest mieux mais 8min de calcul
caracteristiques_selectionnees = ['Valeur_fonciere', 'Surface_reelle_bati', 'Nombre_pieces_principales','Surface_terrain'] # test
X = df[caracteristiques_selectionnees]
                   

y = df['Code_type_local']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


modele_decision_tree = DecisionTreeClassifier(random_state=42)

modele_decision_tree.fit(X_train, y_train)
y_pred = modele_decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9587955600126968


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4084994 entries, 2 to 6521197
Data columns (total 10 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Valeur_fonciere            float64
 1   Commune                    object 
 2   Nombre_de_lots             int64  
 3   Code_type_local            int32  
 4   Surface_reelle_bati        float64
 5   Nombre_pieces_principales  float64
 6   Surface_terrain            float64
 7   day                        object 
 8   month                      object 
 9   year                       object 
dtypes: float64(4), int32(1), int64(1), object(4)
memory usage: 327.2+ MB


In [113]:
# Filling Nans Columns


from sklearn.impute import SimpleImputer

# Supposons que votre DataFrame soit nommé 'df'

# Colonnes à imputer
colonnes_a_imputer = ['Surface_reelle_bati','Nombre_pieces_principales']

# Créez un objet SimpleImputer avec la stratégie 'median'
imputer = SimpleImputer(strategy='median')
imputer.fit(df[colonnes_a_imputer])


# Export Imputer

import pickle as pkl

with open('../model/imputer.pkl','wb') as fichier_pickler:
    pkl.dump(imputer,fichier_pickler)

    
# Appliquez l'imputation sur les colonnes sélectionnées
df[colonnes_a_imputer] = imputer.transform(df[colonnes_a_imputer])


In [114]:
# Removing useless variables
df = df.drop(columns=["Surface_terrain"])

In [115]:
pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)
print(len(df))

df.head()

Valeur_fonciere              0.0
Commune                      0.0
Nombre_de_lots               0.0
Code_type_local              0.0
Surface_reelle_bati          0.0
Nombre_pieces_principales    0.0
day                          0.0
month                        0.0
year                         0.0
dtype: float64
4084994


,Valeur_fonciere,Commune,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,day,month,year
2,45000.0,SAINT DENIS,1,2,20.0,1.0,22,8,2018
3,100000.0,HYERES,2,2,52.0,3.0,13,3,2018
4,102000.0,ORCHIES,0,1,54.0,2.0,12,3,2018
5,297000.0,VINCENNES,2,2,46.0,3.0,28,9,2018
6,74000.0,NANTES,2,2,36.0,2.0,9,6,2018


In [116]:
# Adding Variable 'Prix_moyen_m2'

import numpy as np

# Groupez les données par 'Commune' et calculez la somme de 'Surface_reel' et 'Valeur_fonciere' pour chaque commune
groupe_commune = df.groupby('Commune').agg(
    {'Surface_reelle_bati': 'sum', 'Valeur_fonciere': 'sum'})

# Calculez le prix moyen au mètre carré par commune
groupe_commune['Prix_moyen_m2'] = groupe_commune['Valeur_fonciere'] / \
    groupe_commune['Surface_reelle_bati']


# Fusionnez le DataFrame original avec le DataFrame groupe_commune sur la colonne 'Commune'
df = pd.merge(df, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')



# Remplace les inf par NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Supprime les lignes avec NaN dans 'Prix_moyen_m2'
df = df.dropna(subset=['Prix_moyen_m2'], how='any')

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4084941 entries, 2 to 6521197
Data columns (total 10 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Valeur_fonciere            float64
 1   Commune                    object 
 2   Nombre_de_lots             int64  
 3   Code_type_local            int32  
 4   Surface_reelle_bati        float64
 5   Nombre_pieces_principales  float64
 6   day                        int64  
 7   month                      int64  
 8   year                       int64  
 9   Prix_moyen_m2              float64
dtypes: float64(4), int32(1), int64(4), object(1)
memory usage: 327.2+ MB


In [117]:
# Export final data

df.to_pickle('../data/final_data.pkl')


In [118]:
# Export classif model 

with open('../model/modele_classification.pkl','wb') as fichier_pickler:
    pkl.dump(modele_decision_tree,fichier_pickler)
